In [80]:
import numpy as np
import math

# Create some virtual classes
class base:
    __name=""
    
    def __init__(self,name):
        self.__name=name

    def name(self):
        return self.__name

class data(base):
    def __init__(self,name):
        #base.__init__(self,name)  # super().__init__(name)
        super().__init__(name)
        
class alg(base):
    def __init__(self,name):
        super().__init__(name)

In [43]:
class grade(data):
    __value=0
    __numerical=True
    __gradebook_name=str()
    __letter_grades=["F-","F","F+","D-","D","D+","C-","C","C+","B-","B","B+","A-","A","A+"]
    
    def __init__(self,name,numerical=True,value=None):
        if value:
            if isinstance(value,(int,float)):
                self.__numerical=True
            elif isinstance(value,str):
                self.__numerical=False
            self.set(value)
        else:            
            self.__numerical=numerical
        self.__gradebook_name=name
        #data.__init__(self,name+" Grade Algorithm")    
        super().__init__(name+" Grade Algorithm")

    def set(self,value):
        if isinstance(value,(int,float)) and self.__numerical:
            self.__value=value
        elif isinstance(value,str) and not self.__numerical:
            if value in self.__letter_grades:
                self.__value=value
        else:
            print (self.name()+" Error: Bad Grade.")
            raise Exception
    
    def value(self):
        return self.__value
    
    def numerical(self):
        return self.__numerical
    
    def gradebook_name(self):
        return self.__gradebook_name
    
    def __str__(self):
        return self.__gradebook_name+": "+str(self.__value)

class student(data):
    __id_number=0
    __grades=dict()
    
    def __init__(self,first_name, last_name,id_number):
        self.__id_number=id_number
        self.__grades=dict()
        #data.__init__(self,first_name+" "+last_name+" Student Data")
        super().__init__(first_name+" "+last_name+" Student Data")

    def add_grade(self,a_grade,overwrite=False):
        if overwrite or not a_grade.gradebook_name() in self.__grades:
            self.__grades[a_grade.gradebook_name()]=a_grade
        else:
            print (self.name()+" Error Adding Grade "+a_grade.name()+". Grade already exists.")
            raise Exception

    def id_number(self):
        return self.__id_number
    
    def __getitem__(self,key):
        return self.__grades[key]
    
    def print_grades(self):
        for grade in self.__grades:
            print (self.__grades[grade])

In [44]:
class calculator(alg):    
    def __init__(self,name):
        alg.__init__(self,name)

    def apply(self,a_grade_book):
        raise NotImplementedError

In [81]:
class grade_book(data):
    # New member class to hold arbitrary data associated with the class

    __data=dict()
    __students=dict()
    
    def __init__(self,name):
        super().__init__(name)
        self.__students=dict()
        self.__data=dict()
        
    # New method to access data
    def __getitem__(self,key):
        return self.__data[key]
            
    # New method to add data
    def __setitem__(self, key, value):
        self.__data[key] = value
        
    def add_student(self,a_student):
        self.__students[a_student.id_number()]=a_student

    # New method to allow iterating over students
    def get_students(self):
        return self.__students
    
    def assign_grade(self,key,a_grade):
        the_student=None
        try:
            the_student=self.__students[key]
        except:
            for id in self.__students:
                if key == self.__students[id].name():
                    the_student=self.__students[id]
                    break
        if the_student:
            the_student.add_grade(a_grade)
        else:
            print (self.name()+" Error: Did not find student.")
            
    def apply_calculator(self,a_calculator,**kwargs):
        a_calculator.apply(self,**kwargs)

In [65]:
class uncurved_letter_grade_percent(calculator):
    __grades_definition=[ (.97,"A+"),
                          (.93,"A"),
                          (.9,"A-"),
                          (.87,"B+"),
                          (.83,"B"),
                          (.8,"B-"),
                          (.77,"C+"),
                          (.73,"C"),
                          (.7,"C-"),
                          (.67,"C+"),
                          (.63,"C"),
                          (.6,"C-"),
                          (.57,"F+"),
                          (.53,"F"),
                          (0.,"F-")]
    __max_grade=100.
    __grade_name=str()
    
    def __init__(self,grade_name,max_grade=100.):
        self.__max_grade=max_grade
        self.__grade_name=grade_name
        calculator.__init__(self,
                                  "Uncurved Percent Based Grade Calculator "+self.__grade_name+" Max="+str(self.__max_grade))
        
    def apply(self,a_grade_book,grade_name=None,**kwargs):
        if grade_name:
            pass
        else:
            grade_name=self.__grade_name
            
  
        for k,a_student in a_grade_book.get_students().iteritems():
            a_grade=a_student[grade_name]

            if not a_grade.numerical():
                print (self.name()+ " Error: Did not get a numerical grade as input.")
                raise Exception
    
            percent=a_grade.value()/self.__max_grade
        
            for i,v in enumerate(self.__grades_definition):
                if percent>=v[0]:
                    break
                            
            a_student.add_grade(grade(grade_name+" Letter",value=self.__grades_definition[i][1]))

In [66]:
class mean_std_calculator(calculator):
    def __init__(self):
        calculator.__init__(self,"Mean and Standard Deviation Calculator")
        
    def apply(self,a_grade_book,grade_name,**kwargs):
        grades=list()
        for k,a_student in a_grade_book.get_students().iteritems():
            grades.append(a_student[grade_name].value())
        
        a_grade_book[grade_name+" Mean"] = np.mean(grades)
        a_grade_book[grade_name+" STD"] = math.sqrt(np.var(grades))

In [67]:
f=open("Data1401-Grades.csv","r")
first_line = f.readline().rstrip()
fields=first_line.split(",")

data=list()

line = f.readline().rstrip()
while line:
    data.append(line.split(","))
    line = f.readline().rstrip()

f.close()

In [68]:
data

[['1',
  '10',
  '7',
  '0',
  '10',
  '10',
  '8',
  '10',
  '10',
  '10',
  '14',
  '9',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '11',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '1',
  '9.5',
  '15',
  '9',
  '9',
  '0',
  '9',
  '8',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0'],
 ['1',
  '10',
  '7',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '14',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '11',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '1',
  '0',
  '15',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0'],
 ['1',
  '10',
  '7',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '14',
  '9',
  '10',
  '10',
  '10',
  '7',
  '10',
  '3',
  '6',
  '3',
  '3',
  '0',
  '0',
  '0',
  '0',
  '11',
  '0',
  '0',
  '0',
  '

In [69]:
len(data[0])

55

In [70]:
fields.index("l4_1")

26

In [71]:
data[4][fields.index("l4_1")]

'10'

In [82]:
# Problem 1

# Your solution here.
def csv_reader(file):
    f=open(file,"r")
    first_line = f.readline().rstrip()
    fields=first_line.split(",")
    data=list()
    line = f.readline().rstrip()
    while line:
        data.append(line.split(","))
        line = f.readline().rstrip()
    f.close()
    data_dic = list()
    for i in range(len(data)):
        data_dic.append(dict(zip(fields,data[i])))
    return data_dic

In [83]:
dic_data = csv_reader("Data1401-Grades.csv")

In [84]:
dic_data[5]["l3_5"]

'10'

In [75]:
print(dic_data)

[{'l1_n': '1', 'l1_1': '10', '12_n': '7', 'l2_1': '0', 'l2_2': '10', 'l2_3': '10', 'l2_4': '8', 'l2_5': '10', 'l2_6': '10', 'l2_7': '10', 'l3_n': '14', 'l3_1': '9', 'l3_2': '0', 'l3_3': '0', 'l3_4': '0', 'l3_5': '0', 'l3_6': '0', 'l3_7': '0', 'l3_8': '0', 'l3_9': '0', 'l3_10': '0', 'l3_11': '0', 'l3_12': '0', 'l3_13': '0', 'l3_14': '0', 'l4_n': '11', 'l4_1': '0', 'l4_2': '0', 'l4_3': '0', 'l4_4': '0', 'l4_5': '0', 'l4_6': '0', 'l4_7': '0', 'l4_8': '0', 'l4_9': '0', 'l4_10': '0', 'l4_11': '0', 'q1_n': '1', 'q1_1': '9.5', 'e1_n': '15', 'e1_1': '9', 'e1_2': '9', 'e1_3': '0', 'e1_4': '9', 'e1_5': '8', 'e1_6': '0', 'e1_7': '0', 'e1_8': '0', 'e1_9': '0', 'e1_10': '0', 'e1_11': '0', 'e1_12': '0', 'e1_13': '0', 'e1_14': '0', 'e1_15': '0'}, {'l1_n': '1', 'l1_1': '10', '12_n': '7', 'l2_1': '0', 'l2_2': '0', 'l2_3': '0', 'l2_4': '0', 'l2_5': '0', 'l2_6': '0', 'l2_7': '0', 'l3_n': '14', 'l3_1': '0', 'l3_2': '0', 'l3_3': '0', 'l3_4': '0', 'l3_5': '0', 'l3_6': '0', 'l3_7': '0', 'l3_8': '0', 'l3_9': 

In [85]:
print(dic_data[5]['l3_4'])   # first index is example number and second index is key

8


In [95]:
#  Problem 2

# Creating a grade book

dic_data = csv_reader("Data1401-Grades.csv")
a_grade_book=grade_book("Gradebook")
for student_i in range(len(dic_data)):
    a_student_0=student("Student",str(student_i),student_i)
    for k in dic_data[student_i].keys():
        a_student_0.add_grade(grade(k,value=dic_data[student_i][k]))
        a_grade_book.add_student(a_student_0)

In [96]:
for i in a_grade_book.name():
    print(i)

G
r
a
d
e
b
o
o
k


In [79]:
class_data=csv_reader("Data1401-Grades.csv")


In [87]:
for k in class_data[10].keys():
    print(k)

l1_n
l1_1
12_n
l2_1
l2_2
l2_3
l2_4
l2_5
l2_6
l2_7
l3_n
l3_1
l3_2
l3_3
l3_4
l3_5
l3_6
l3_7
l3_8
l3_9
l3_10
l3_11
l3_12
l3_13
l3_14
l4_n
l4_1
l4_2
l4_3
l4_4
l4_5
l4_6
l4_7
l4_8
l4_9
l4_10
l4_11
q1_n
q1_1
e1_n
e1_1
e1_2
e1_3
e1_4
e1_5
e1_6
e1_7
e1_8
e1_9
e1_10
e1_11
e1_12
e1_13
e1_14
e1_15
